In [16]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
import warnings
import mlflow
from modeling.config import EXPERIMENT_NAME
TRACKING_URI = open("../.mlflow_uri").read().strip()

ROOT = os.environ.get('PWD')

warnings.filterwarnings('ignore')







In [ ]:
# our data, read into dataframe df_master_tfw
df_master_tfw = pd.read_csv('../data/master_no_missing_20210719.csv')

In [ ]:
list(df_master_tfw.columns.values)

# Data cleaning and feature engineering

In [ ]:
del df_master_tfw['Unnamed: 0']

In [ ]:
list(df_master_tfw.columns.values)

In [15]:
# set display default so that we can see all the columns we have in play

pd.set_option('display.max_rows', 10)

In [ ]:
# convert to datetime, this list can be varied depending on what stays in for a particular model

dates = ['arrival_date', 'departure_date', 'date']

for i in dates:
    df_master_tfw[i] = pd.to_datetime(df_master_tfw[i])

In [ ]:
# for our first model, let#s drop most of the columns we tried to keep

drop_list = ['binding_inquiry',
'phone_clicks',
'library',
"children`s_room",
'corridor',
'gallery',
'heating_room',
'lounge',
'storage_room',
'washroom',
'wellness',
'winter_garden',
'Blu-ray_player',
'CDs_DVDs',                               
'DVD-player',                             
'VCR',
'alarm_system',                            
'armchair',                             
'awning_',                                
'baby_monitor',                           
'bath_towels',                            
'bathrobe',
'beach_chair',
'bidet',                                    
'boat',                                    
'boat_dock',                                
'books',                                   
'bottle_warmer',                           
'bread_service',                            
'bunk_bed',                               
'carport',                                  
'central_heating',                       
'changing_mat',                            
'chest_of_drawers',
"child's_bed",                              
'children_toilet_seat',                    
'cleaning_supplies',
'colouring_book_pencils',
'crockery',                                
'darts',                                   
'desk',                                    
'dining_table',
'double_wash_basin',                        
'drying_rack',                             
'egg_cooker',
'exercise_room',
'fax',                                     
'fire_alarm',
'first-aid_kit',                            
'flat_iron',                               
'fly_screen',                              
'folding_bed',
'food_processor',
'game_console',
'garden_shed',                             
'guest_toilet', 
'high_chair',
'hot_water',
'ironing_board',                           
'kettle',
'laundry_service',                         
'lawn',                                    
'loft_bed',                                
'make-up_mirror',
'mirror',                                  
'mixer',                                   
'outdoor_shower',                          
'phone',                                   
'piano',                                   
'pier',                                    
'playground',                              
'pond',
'relaxation_room',                         
'room_service',                            
'safe',                                     
'sandpit',                                 
'sandwich_toaster',
'shoe_cabinet', 
'ski_boot_heaters',                        
'slide',                                   
'socket_covers',                           
'sofa',
'spices',                                  
'stair_gate',
'stereo_system',                          
'sun_loungers',                            
'sun_umbrella_',                           
'sunshade',
'shared_swimming_pool',
'swimming_pool',                           
'swing',                                   
'table_football',
'table_tennis',
'tea_towels',                              
'toaster',                                 
'toilet',                                  
'towels',                                  
'toys',
'trampoline',                              
'underfloor_heating',                      
'vacuum_cleaner',                          
'walk-in_shower',                           
'wall_bed',
'wardrobe',                                 
'wash_basin',                              
'water_bed',                               
'windbreak',
'title',
'contract_end',
'region',
'holiday_region',
'zip']


In [ ]:
tiny_master = df_master_tfw.drop(drop_list,axis=1)

In [ ]:
# saving to csv

tiny_master.to_csv('../data/tiny_master.csv')

In [17]:
# read from csv

tiny_master = pd.read_csv('../data/tiny_master.csv')

In [18]:
# convert to datetime, this list can be varied depending on what stays in for a particular model

dates = ['arrival_date', 'departure_date', 'date']

for i in dates:
    tiny_master[i] = pd.to_datetime(tiny_master[i])

In [ ]:
tiny_master['month_year'] = tiny_master['arrival_date'].dt.to_period('M')

In [19]:
tiny_master['year'] = pd.DatetimeIndex(tiny_master['arrival_date']).year
tiny_master['month'] = pd.DatetimeIndex(tiny_master['arrival_date']).month

In [20]:
# extract the data for arrival date in 2019 and 2020 only

tiny_master_19 = tiny_master.query("year == 2019")

In [21]:
import math 

# We normalize x values to match with the 0-2π cycle
tiny_master_19["month_norm"] = 2 * math.pi * tiny_master_19["month"] / tiny_master_19["month"].max()

tiny_master_19["cos_month"] = np.cos(tiny_master_19["month_norm"])
tiny_master_19["sin_month"] = np.sin(tiny_master_19["month_norm"])

In [23]:
# we calculate price per day from the inquiry price and length of day

tiny_master_19['price_per_day'] = (tiny_master_19['inquiry_price']/tiny_master_19['length_stay'])

In [32]:
tiny_master_19.head()

,Unnamed: 0,listing_id,result_views,expose_views,1_room_apartment,dining_room,en_suite_bathroom,kitchen,kitchen_living,living_bedroom,...,bathrooms,bedrooms,max_guests,living_area,year,month,month_norm,cos_month,sin_month,price_per_day
0,0,97232bc1-cee6-54cc-9965-be13177051d3,56214,12533,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,2.0,4.0,60.0,2019,1,0.523599,0.866025,5.000000e-01,59.0
1,1,97232bc1-cee6-54cc-9965-be13177051d3,56214,12533,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,2.0,4.0,60.0,2019,2,1.047198,0.500000,8.660254e-01,79.0
2,2,97232bc1-cee6-54cc-9965-be13177051d3,56214,12533,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,2.0,4.0,60.0,2019,12,6.283185,1.000000,-2.449294e-16,139.0
3,3,97232bc1-cee6-54cc-9965-be13177051d3,56214,12533,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,2.0,4.0,60.0,2019,12,6.283185,1.000000,-2.449294e-16,139.0
4,4,97232bc1-cee6-54cc-9965-be13177051d3,56214,12533,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,2.0,4.0,60.0,2019,1,0.523599,0.866025,5.000000e-01,59.0


In [25]:
# create dataframe from the aggregate data

tiny_master_agg_price = tiny_master_19.groupby(['listing_id','month']).agg(['mean','max','min'])['price_per_day'].reset_index()

In [26]:
# check

tiny_master_agg_price.head()

,listing_id,month,mean,max,min
0,00005de7-6ed8-541b-be83-2573a46cd307,12,125.000000,125.0,125.000000
1,0003f153-450a-5837-8a46-bd5db5b536b1,1,45.000000,45.0,45.000000
2,0003f153-450a-5837-8a46-bd5db5b536b1,2,52.833333,57.5,44.666667
3,0003f153-450a-5837-8a46-bd5db5b536b1,3,68.611111,85.0,46.666667
4,0003f153-450a-5837-8a46-bd5db5b536b1,4,60.612745,67.5,47.750000


In [33]:
medium_master = pd.merge(tiny_master_agg_price, tiny_master_19, left_on=['listing_id','month'], right_on=['listing_id','month'], how='right')

In [35]:
medium_master.head(3)

,listing_id,month,mean,max,min,Unnamed: 0,result_views,expose_views,1_room_apartment,dining_room,...,balcony,bathrooms,bedrooms,max_guests,living_area,year,month_norm,cos_month,sin_month,price_per_day
0,97232bc1-cee6-54cc-9965-be13177051d3,1,58.500000,59.0,57.0,0,56214,12533,0.0,0.0,...,1,1.0,2.0,4.0,60.0,2019,0.523599,0.866025,5.000000e-01,59.0
1,97232bc1-cee6-54cc-9965-be13177051d3,2,76.714286,81.0,74.0,1,56214,12533,0.0,0.0,...,1,1.0,2.0,4.0,60.0,2019,1.047198,0.500000,8.660254e-01,79.0
2,97232bc1-cee6-54cc-9965-be13177051d3,12,114.153846,139.0,55.0,2,56214,12533,0.0,0.0,...,1,1.0,2.0,4.0,60.0,2019,6.283185,1.000000,-2.449294e-16,139.0


In [42]:
# fold in monthly inquiry counts

ju_hu = medium_master.groupby(['listing_id','month']).agg(['count'])['price_per_day'].reset_index()

In [43]:
# merging the new master dataset

master = pd.merge(ju_hu, medium_master, left_on=['listing_id','month'], right_on=['listing_id','month'], how='right')

In [50]:
master.drop(['month','Unnamed: 0','month_norm'],axis=1,inplace=True)

In [48]:
master.rename(columns={'count':'inquiry_count','mean':'mean_price_per_day','max':'max_price_per_day','min':'min_price_per_day'},inplace=True)

In [53]:
master.head()

,listing_id,inquiry_count,mean_price_per_day,max_price_per_day,min_price_per_day,result_views,expose_views,1_room_apartment,dining_room,en_suite_bathroom,...,terrace,balcony,bathrooms,bedrooms,max_guests,living_area,year,cos_month,sin_month,price_per_day
0,97232bc1-cee6-54cc-9965-be13177051d3,4,58.500000,59.0,57.0,56214,12533,0.0,0.0,0.0,...,0,1,1.0,2.0,4.0,60.0,2019,0.866025,5.000000e-01,59.0
1,97232bc1-cee6-54cc-9965-be13177051d3,7,76.714286,81.0,74.0,56214,12533,0.0,0.0,0.0,...,0,1,1.0,2.0,4.0,60.0,2019,0.500000,8.660254e-01,79.0
2,97232bc1-cee6-54cc-9965-be13177051d3,13,114.153846,139.0,55.0,56214,12533,0.0,0.0,0.0,...,0,1,1.0,2.0,4.0,60.0,2019,1.000000,-2.449294e-16,139.0
3,97232bc1-cee6-54cc-9965-be13177051d3,13,114.153846,139.0,55.0,56214,12533,0.0,0.0,0.0,...,0,1,1.0,2.0,4.0,60.0,2019,1.000000,-2.449294e-16,139.0
4,97232bc1-cee6-54cc-9965-be13177051d3,4,58.500000,59.0,57.0,56214,12533,0.0,0.0,0.0,...,0,1,1.0,2.0,4.0,60.0,2019,0.866025,5.000000e-01,59.0


In [56]:
# drop these columns for the purpose of the model

master.drop(['result_views','expose_views'],axis=1,inplace=True)

In [57]:
# drop these columns for the purpose of the model

master.drop(['date','arrival_date','departure_date'],axis=1,inplace=True)

In [59]:
# drop these columns for the purpose of the model

master.drop(['inquiry_price','price_per_day'],axis=1,inplace=True)

In [61]:
'year'
master.drop(['year'],axis=1,inplace=True)

In [60]:
list(master.columns.values)

['listing_id',
 'inquiry_count',
 'mean_price_per_day',
 'max_price_per_day',
 'min_price_per_day',
 '1_room_apartment',
 'dining_room',
 'en_suite_bathroom',
 'kitchen',
 'kitchen_living',
 'living_bedroom',
 'living_/_dining_room',
 'living_room',
 'roof-deck',
 'separate_WC',
 'veranda',
 'additional_bed',
 'bathtub',
 'bed_linen',
 'bicycles',
 'coffee_machine',
 'computer',
 'double_bed',
 'fan',
 'fireplace',
 'freezer',
 'games',
 'garage',
 'garden_furniture',
 'hair_dryer',
 'hot_tub',
 'infrared_cabin',
 'kingsize_bed',
 'microwave',
 'pool_table',
 'private_parking',
 'radio',
 'refrigerator',
 'shower',
 'single_bed',
 'sofa_bed',
 'steam_bath',
 'tanning_bed',
 'cooking',
 'adult_count',
 'children_count',
 'pets_count',
 'length_stay',
 'subregion',
 'property_type',
 'option_allergic',
 'option_non_smoking_only',
 'option_holiday_with_your_pet',
 'option_holiday_with_your_horse',
 'option_holiday_with_your_dog',
 'option_close_to_the_beach',
 'option_wheelchair_accessibl

In [62]:
# trying not to be a dummy by getting dummies

master_dummies = pd.get_dummies(master, columns=['subregion','property_type'])

In [63]:
master_dummies.to_csv('../data/master_with_dummies.csv')

In [64]:
master_dummies.shape

(418124, 274)

In [79]:
X = master_dummies.drop(['listing_id','inquiry_count'],axis=1)
Y = master_dummies['inquiry_count']

In [80]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

In [85]:
# we love to normalize our data!!

from sklearn.preprocessing import MinMaxScaler


In [82]:
the_list = ['adult_count','children_count','pets_count','length_stay','bathrooms','bedrooms','max_guests','living_area','mean_price_per_day','max_price_per_day','min_price_per_day']

In [83]:
# Scaling with MinMaxScaler
scaler_norm = MinMaxScaler()
X_train_scaled_norm = scaler_norm.fit_transform(X_train[the_list])
X_test_scaled_norm = scaler_norm.transform(X_test[the_list])

In [84]:
# Concatenating normalized columns 
X_train_preprocessed_norm = np.concatenate([X_train_scaled_norm, X_train.drop(the_list, axis=1)], axis=1)
X_test_preprocessed_norm = np.concatenate([X_test_scaled_norm, X_test.drop(the_list, axis=1)], axis=1)

In [86]:
X_train_preprocessed_norm.shape

(292686, 272)

In [87]:
X_test_preprocessed_norm.shape

(125438, 272)

## Trainining the model and tracking with MLFlow

In [75]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.start_run()
run = mlflow.active_run()

In [76]:
print("Active run_id: {}".format(run.info.run_id))

Active run_id: c00b4d522d9c411c8ee972898f839110


In [77]:
master_dummies.isna().sum()

listing_id                         0
inquiry_count                      0
mean_price_per_day                 0
max_price_per_day                  0
min_price_per_day                  0
                                  ..
subregion_Überlinger See           0
property_type_apartment            0
property_type_bungalow             0
property_type_holiday_apartment    0
property_type_holiday_houses       0
Length: 274, dtype: int64

In [88]:
#training the model
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)

In [89]:
# we will use root mean square error as metric

from sklearn.metrics import mean_squared_error
y_train_pred = reg.predict(X_train)
rmse_train = mean_squared_error(y_train, y_train_pred,squared=False)
print(rmse_train)

24.06275338056806


In [91]:
# let's see what the test set does (boom!)

y_test_pred = reg.predict(X_test)
rmse_test = mean_squared_error(y_test, y_test_pred,squared=False)
print(rmse_test)

24.417879789542205


In [97]:
import shap

In [98]:
X100 = shap.utils.sample(X, 100) # 100 instances for use as the background distribution

In [100]:
print("Model coefficients:\n")
for i in range(X.shape[1]):
    print(X.columns[i], "=", reg.coef_[i].round(4))

Model coefficients:

mean_price_per_day = 0.1009
max_price_per_day = 0.0635
min_price_per_day = -0.1791
1_room_apartment = -2.4048
dining_room = 1.3317
en_suite_bathroom = -2.8281
kitchen = 2.3668
kitchen_living = 0.0659
living_bedroom = -1.0583
living_/_dining_room = -3.0165
living_room = -0.2164
roof-deck = 13.141
separate_WC = 0.4454
veranda = 0.0912
additional_bed = -3.8931
bathtub = 0.4443
bed_linen = 3.56
bicycles = -2.3069
coffee_machine = -2.8144
computer = -1.4213
double_bed = -0.0021
fan = 1.2294
fireplace = -0.9119
freezer = 0.9864
games = -3.48
garage = -1.2237
garden_furniture = 12.7094
hair_dryer = -0.5595
hot_tub = 1.2964
infrared_cabin = 2.7038
kingsize_bed = -2.9557
microwave = -0.2782
pool_table = -5.1934
private_parking = -1.24
radio = 1.2672
refrigerator = 3.9081
shower = 1.0706
single_bed = -0.9266
sofa_bed = 1.199
steam_bath = 4.9359
tanning_bed = 11.0514
cooking = -2.6865
adult_count = 0.5825
children_count = 2.2796
pets_count = -3.0969
length_stay = 0.1785
optio

In [101]:

explainer = shap.Explainer(reg.predict, X100)
shap_values = explainer(X)

ValueError: Could not find feature named: RM

In [93]:
params = {
      "target": master['inquiry_count'],
      "fit_intercept": True,
  }

In [94]:
mlflow.log_params(params)
mlflow.set_tag("running_from_visual_studio_code", "our baseline model")
mlflow.log_metric("train -" + "RMSE", rmse_train)
mlflow.log_metric("test -" + "RMSE", rmse_test)
# mlflow.log_artifact("../models")
# mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [95]:
mlflow.get_run(run_id=run.info.run_id)

<Run: data=<RunData: metrics={'test -RMSE': 24.417879789542205, 'train -RMSE': 24.06275338056806}, params={'fit_intercept': 'True',
 'target': '0          4\n'
           '1          7\n'
           '2         13\n'
           '3         13\n'
           '4          4\n'
           '          ..\n'
           '418119    11\n'
           '418120    11\n'
           '418121     1\n'
           '418122     1\n'
           '418123     1\n'
           'Name: inquiry_count, Length: 418124, dtype: int64'}, tags={'mlflow.source.name': 'ipykernel_launcher',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'petermcisaac',
 'running_from_visual_studio_code': 'our baseline model'}>, info=<RunInfo: artifact_uri='s3://neuefische-mlflow/mlflow-artifacts/25/c00b4d522d9c411c8ee972898f839110/artifacts', end_time=1626789340971, experiment_id='25', lifecycle_stage='active', run_id='c00b4d522d9c411c8ee972898f839110', run_uuid='c00b4d522d9c411c8ee972898f839110', start_time=1626788362298, status='FINISHED', u

## Checking the experiments

while the next cell is running you will not be able to run other cells in the notebook

In [ ]:
!mlflow ui